In [54]:
import json, re, nltk

import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import pandas as pd

import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from nltk.stem.wordnet import WordNetLemmatizer

stopwords = nltk.corpus.stopwords.words('english')
podcast_stopwords = ['support', 'podcast', 'https', 'http', '', 'follow', 'us', 'on', 'instagram', 'twitter', 'youtube', 'facebook']
stopwords += podcast_stopwords

In [55]:
raw_dat = pd.read_csv('../script_output/episode_transcript_data_w_metadata.csv')
raw_dat.head()

,index,show_uri,show_name,show_description,publisher,language,rss_link,episode_uri,episode_name,episode_description,duration,show_filename_prefix,episode_filename_prefix,Unnamed: 0,episode,transcript
0,0,spotify:show:2NYtxEZyYelR6RMKmjfPLB,Kream in your Koffee,A 20-something blunt female takes on the world...,Katie Houle,['en'],https://anchor.fm/s/11b84b68/podcast/rss,spotify:episode:000A9sRBYdVh66csG2qEdj,1: It’s Christmas Time!,On the first ever episode of Kream in your Kof...,12.700133,show_2NYtxEZyYelR6RMKmjfPLB,000A9sRBYdVh66csG2qEdj,34866,000A9sRBYdVh66csG2qEdj,Hello. Hello. Hello everyone. This is Katie an...
1,1,spotify:show:15iWCbU7QoO23EndPEO6aN,Morning Cup Of Murder,Ever wonder what murder took place on today in...,Morning Cup Of Murder,['en'],https://anchor.fm/s/b07181c/podcast/rss,spotify:episode:000HP8n3hNIfglT2wSI2cA,The Goleta Postal Facility shootings- January ...,"See something, say something. It’s a mantra ma...",6.019383,show_15iWCbU7QoO23EndPEO6aN,000HP8n3hNIfglT2wSI2cA,14162,000HP8n3hNIfglT2wSI2cA,There were two more murders 15 miles away arri...
2,2,spotify:show:6vZRgUFTYwbAA79UNCADr4,Inside The 18 : A Podcast for Goalkeepers by G...,Inside the 18 is your source for all things Go...,Inside the 18 GK Media,['en'],https://anchor.fm/s/81a072c/podcast/rss,spotify:episode:001UfOruzkA3Bn1SPjcdfa,Ep.36 - Incorporating a Singular Goalkeeping C...,Today’s episode is a sit down Michael and Omar...,43.616333,show_6vZRgUFTYwbAA79UNCADr4,001UfOruzkA3Bn1SPjcdfa,93168,001UfOruzkA3Bn1SPjcdfa,Welcome to inside the 18. Today's episode is t...
3,3,spotify:show:5BvKEjaMSuvUsGROGi2S7s,Arrowhead Live!,Your favorite podcast for everything @Chiefs! ...,Arrowhead Live!,['en-US'],https://anchor.fm/s/917dba4/podcast/rss,spotify:episode:001i89SvIQgDuuyC53hfBm,Episode 1: Arrowhead Live! Debut,Join us as we take a look at all current Chief...,58.189200,show_5BvKEjaMSuvUsGROGi2S7s,001i89SvIQgDuuyC53hfBm,69703,001i89SvIQgDuuyC53hfBm,Hey cheese fans before we get started. I wante...
4,4,spotify:show:7w3h3umpH74veEJcbE6xf4,FBoL,"The comedy podcast about toxic characters, wri...",Emily Edwards,['en'],https://www.fuckboisoflit.com/episodes?format=rss,spotify:episode:0025RWNwe2lnp6HcnfzwzG,"The Lion, The Witch, And The Wardrobe - Ashley...",The modern morality tail of how to stay good f...,51.782050,show_7w3h3umpH74veEJcbE6xf4,0025RWNwe2lnp6HcnfzwzG,104381,0025RWNwe2lnp6HcnfzwzG,"Sorry to interrupt the show, but I do have to ..."


In [56]:
raw_dat = raw_dat.dropna()

In [57]:
raw_dat['episode_description_cleaned'] = raw_dat['episode_description'].str.lower()
# raw_dat['show_description_cleaned'] = raw_dat['show_description'].str.lower()
# raw_dat['show_episode_description_concat'] = raw_dat['episode_description_cleaned'] + raw_dat['show_description_cleaned']
noise = ['@', 'www', 'http', 'discord.gg', 'social media:', 'on social media', 'new episodes', 'latest episodes', 'episode', 'anchor']
for n in noise: 
    raw_dat['episode_description_cleaned'] = raw_dat['episode_description_cleaned'].str.replace(f'\S*{n}\S*\s?', '', regex=True)


In [58]:
def tokenize_titles(title):
    tokens = nltk.word_tokenize(title)
    lmtzr = WordNetLemmatizer()
    filtered_tokens = []
    
    for token in tokens:
        token = token.lower()
        # print(token)
        token = token.replace("'s", " ").replace("n’t", " not").replace("’ve", " have")
        token = re.sub(r'[^a-zA-Z]', '', token)
        token = re.sub(r'\b\w*bitly\w*\b', '', token)
        token = re.sub(r'\b\w*com\b', '', token)

        if token not in stopwords:
            filtered_tokens.append(token.lower())       
    
    lemmas = [lmtzr.lemmatize(t,'v') for t in filtered_tokens]

    return lemmas

In [59]:
# n = 3
# print(raw_dat.show_description[n])
# print(raw_dat.show_description_cleaned[n])
# tokenize_titles(raw_dat.show_description_cleaned[n])


In [60]:
raw_dat['show_description_tokens'] = raw_dat.episode_description_cleaned.apply(tokenize_titles)
raw_dat['show_description_preprocessed'] = raw_dat.show_description_tokens.str.join(' ')

In [66]:
tf_vectorizer = CountVectorizer(strip_accents = 'unicode',
                                stop_words = 'english',
                                lowercase = True,
                                # tokenizer=tokenize_titles,
                                max_features=500,
                                token_pattern = r'\b[a-zA-Z]{3,}\b',
                                max_df = 0.8, 
                                min_df = 20,
                                ngram_range=(2,4))
dtm_tf = tf_vectorizer.fit_transform(raw_dat.show_description_preprocessed)
print(dtm_tf.shape)

(105153, 500)


In [67]:
lda_tf = LatentDirichletAllocation(n_components=5, 
                                   max_iter=50,
                                   learning_method='online', 
                                   random_state = 0, 
                                   n_jobs=-1)
lda_tf.fit(dtm_tf)

LatentDirichletAllocation(learning_method='online', max_iter=50, n_components=5,
                          n_jobs=-1, random_state=0)

In [68]:
n_top_words = 20
tf_feature_names = tf_vectorizer.get_feature_names()

topics = dict()
for topic_idx, topic in enumerate(lda_tf.components_):
    topics[topic_idx] = [tf_feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]
    print("Topic #%d:" % topic_idx)
    print(" | ".join([tf_feature_names[i] for i in topic.argsort()[:-n_top_words - 1:-1]]))
    print("\n")

Topic #0:
voice message | send voice | send voice message | make send | make send voice message | make send voice | easiest way make send | way make send | way make send voice | thank listen | mental health | rate review | apple podcast | daily fortnite | answer question | today talk | want hear | review subscribe | tip trick | unite state


Topic #1:
way make | easiest way | easiest way make | anchor easiest | anchor easiest way | anchor easiest way make | sponsor anchor | sponsor anchor easiest | sponsor anchor easiest way | feel like | share story | leave star | week talk | star review | thank listen | answer question | high school | stay tune | send email | unite state


Topic #2:
social media | make sure | hope enjoy | let know | new york | special guest | leave review | new year | promo code | ideal prepare | info visit | playlist explore | exam info visit | exam info | merch sponsor | merch sponsor anchor | merch sponsor anchor easiest | like buy | buy merch | sure subscribe


T

In [69]:
# content = raw_dat[raw_dat['show_description_preprocessed'].str.contains('instagram')]['show_description_preprocessed'].tolist()

# for i in content: 
#     print(i, '\n')

In [70]:
pyLDAvis.sklearn.prepare(lda_tf, dtm_tf, tf_vectorizer)

/Users/christianwiloejo/opt/anaconda3/lib/python3.9/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  default_term_info = default_term_info.sort_values(
/Users/christianwiloejo/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/christianwiloejo/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reload
/Users/christianwiloejo/opt/anaconda3/lib/python3.9/site-packages/past/builtins/misc.py:45: DeprecationWarning: the imp module is deprecated in favour of importlib; see the module's documentation for alternative uses
  from imp import reloa

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
4      0.248788 -0.057367       1        1  35.741317
1     -0.301995 -0.203403       2        1  29.713306
3     -0.160729  0.400171       3        1  18.341149
0     -0.033999 -0.183472       4        1  11.127784
2      0.247936  0.044071       5        1   5.076445, topic_info=                         Term          Freq         Total Category  logprob  \
490                  way make  23378.000000  23378.000000  Default  30.0000   
139               easiest way  23328.000000  23328.000000  Default  29.0000   
140          easiest way make  23290.000000  23290.000000  Default  28.0000   
20         anchor easiest way  23289.000000  23289.000000  Default  27.0000   
19             anchor easiest  23289.000000  23289.000000  Default  26.0000   
..                        ...           ...           ...      ...      ...   
96   buy merch sponsor anchor    735.479294    736.279904   Topic5  -3.8999   
264    like buy merch sponsor    735.479294    736.279904   Topic5  -3.8999   
291                 love hear    664.323461    665.127967   Topic5  -4.0016   
467                 use promo    642.294814    643.096557   Topic5  -4.0354   
468            use promo code    639.262413    640.064166   Topic5  -4.0401   

     loglift  
490  30.0000  
139  29.0000  
140  28.0000  
20   27.0000  
19   26.0000  
..       ...  
96    2.9795  
264   2.9795  
291   2.9793  
467   2.9793  
468   2.9793  

[194 rows x 6 columns], token_table=      Topic      Freq                     Term
term                                          
9         3  0.999744          amazon purchase
18        4  0.998422               anchor app
19        2  0.999945           anchor easiest
20        2  0.999945       anchor easiest way
21        2  0.999976  anchor easiest way make
...     ...       ...                      ...
489       4  0.999601                want hear
490       2  0.999976                 way make
491       4  0.999723            way make send
492       4  0.999917      way make send voice
493       2  0.999252                week talk

[158 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 2, 4, 1, 3])